# Data Cleaning - Bases de données patient, consommant et hospitalisation

### Installation libraire requise

In [1]:
%%capture capt
!pip install pandas_profiling

### Import librairies

In [2]:
%%capture capt
import numpy as np
import os
import pandas as pd
from pandas_profiling import ProfileReport
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore")

### Import des bases de données

In [3]:
hospit=pd.read_csv("base_hospit.csv", sep=';')
patient=pd.read_csv("base_patient.csv", sep=';')
consommant=pd.read_csv("consommant.csv", sep=';')

### Function utile

In [4]:
def count():
    print(f"Nb patients = {len(patient)}")
    print(f"Nb hospit = {len(hospit)}")
    print(f"Nb conso = {len(consommant)}")

### Conversion des dates en datetime

In [5]:
hospit['EXE_SOI_DTD']=pd.to_datetime(hospit['EXE_SOI_DTD'], dayfirst=True)
hospit['SEJ_NBJ']=hospit['SEJ_NBJ'].fillna(np.nan).astype(float)
hospit['SEJ_NBJ']=hospit['SEJ_NBJ'].astype('Int64')

In [6]:
consommant['datemax']=pd.to_datetime(consommant['datemax'], dayfirst=True)

In [7]:
patient['date_h0']=pd.to_datetime(patient['date_h0'], dayfirst=True)
patient['dte_deces']=pd.to_datetime(patient['dte_deces'], dayfirst=True)

## Encodage des patients

In [8]:
dict_patients = {}
for i in range(len(patient)):
    code = patient.CODE_PATIENT.iloc[i]
    dict_patients[code] = i

In [9]:
patient['CODE_PATIENT_ENC'] = ['P' + str(i) for i in range(len(patient))]

In [10]:
enc_conso = []
for i in range(len(consommant)):
    code = consommant.CODE_PATIENT.iloc[i]
    patient_n = dict_patients[code]
    enc_conso.append('P'+str(patient_n))
consommant['CODE_PATIENT_ENC'] = enc_conso

In [11]:
enc_hospit = []
for i in range(len(hospit)):
    code = hospit.BEN_NIR_IDT.iloc[i]
    patient_n = dict_patients[code]
    enc_hospit.append('P' + str(patient_n))
hospit['BEN_NIR_IDT_ENC'] = enc_hospit

In [12]:
patient=patient.drop(columns=['CODE_PATIENT'])
consommant=consommant.drop(columns=['CODE_PATIENT'])
hospit=hospit.drop(columns=['BEN_NIR_IDT'])

In [ ]:
import pickle
    
with open('dict_codes_patients.pickle', 'wb') as handle:
    pickle.dump(dict_patients, handle)

In [13]:
patient=patient.rename(columns={'CODE_PATIENT_ENC':'CODE_PATIENT'})
consommant=consommant.rename(columns={'CODE_PATIENT_ENC':'CODE_PATIENT'})
hospit=hospit.rename(columns={'BEN_NIR_IDT_ENC':'BEN_NIR_IDT'})

## Suppression des patients dont la première hospitalisation est postérieure à 2016 ou antérieure à 2010

In [14]:
patient=patient[(patient['date_h0']<'01-01-2016')&(patient['date_h0']>='01-01-2010')]

In [15]:
patient['date_h0']=pd.to_datetime(patient['date_h0'], dayfirst=True)

In [16]:
consommant = consommant[consommant['CODE_PATIENT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [17]:
consommant.head()

,_TEMG001,datemax,CODE_PATIENT
0,21769,2019-08-08,P0
1,20150,2015-03-03,P6
2,20312,2015-08-12,P8
3,21545,2018-12-27,P10
4,21769,2019-08-08,P13


In [18]:
hospit.head()

,RSA_NUM,ETA_NUM,EXE_SOI_DTD,SEJ_NBJ,GRG_GHM,DGN_PAL,DGN_REL,BEN_NIR_IDT
0,144689,310781406,2011-11-22,2,05K051,I2100,NaN,P0
1,2763,110780061,2011-11-24,8,05M042,I21000,NaN,P0
2,15548,110780061,2011-12-08,0,05M16T,I255,NaN,P0
3,5260,110780061,2012-03-04,1,05M09T,I501,NaN,P0
4,112258,310781406,2012-03-05,7,05M092,I501,NaN,P0


In [19]:
hospit = hospit[hospit['BEN_NIR_IDT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

### Ajout des morts et durée de survie

In [20]:
patient['Mort'] = [not pd.isnull(patient.iloc[i].dte_deces) for i in range(len(patient))]

In [21]:
patient['Nb_survie'] = (patient.dte_deces - patient.date_h0).apply(lambda x: x if x.days >=0 else 1)

In [22]:
nb_survie = []
for i in range(len(patient)):
    if patient.iloc[i].Mort:
        survie_patient = patient.dte_deces.iloc[i] - patient.date_h0.iloc[i]
        if survie_patient.days < 0:
            nb_survie.append(1)
        else:
            nb_survie.append(survie_patient)
    else:
        nb_survie.append(pd.to_datetime('31/12/2018') - patient.date_h0.iloc[i])

In [23]:
patient['Nb_survie'] = nb_survie

In [24]:
patient.Nb_survie = patient.Nb_survie.apply(lambda x: x.days if x != 1 else 1)

###### On ajoute les durées de survie et la mort de chaque patient sur la base des hospitalisations

In [25]:
patient_mort=patient.loc[patient["Mort"]]
code_patient_mort = set(patient_mort.CODE_PATIENT)

hospit["Mort"] = [False]*len(hospit)
hospit.loc[hospit.BEN_NIR_IDT.isin(code_patient_mort), "Mort"] = True

In [26]:
dict_survie = {}
for i in range(len(patient)):
    dict_survie[patient.CODE_PATIENT.iloc[i]] = patient.Nb_survie.iloc[i]

survies = []
for i in range(len(hospit)):
    code_patient = hospit.BEN_NIR_IDT.iloc[i]
    survies.append(dict_survie[code_patient])
    
hospit["Nb_survie"] = survies

In [27]:
hospit

,RSA_NUM,ETA_NUM,EXE_SOI_DTD,SEJ_NBJ,GRG_GHM,DGN_PAL,DGN_REL,BEN_NIR_IDT,Mort,Nb_survie
0,144689,310781406,2011-11-22,2,05K051,I2100,NaN,P0,False,2596
1,2763,110780061,2011-11-24,8,05M042,I21000,NaN,P0,False,2596
2,15548,110780061,2011-12-08,0,05M16T,I255,NaN,P0,False,2596
3,5260,110780061,2012-03-04,1,05M09T,I501,NaN,P0,False,2596
4,112258,310781406,2012-03-05,7,05M092,I501,NaN,P0,False,2596
...,...,...,...,...,...,...,...,...,...,...
158710,16969,130784713,2013-02-21,30,04M174,J91,NaN,P24308,True,484
158711,15396,810000331,2010-12-18,17,08C322,S8221,NaN,P24310,False,2935
158712,4738,810000331,2011-01-07,0,08C14J,Z470,NaN,P24310,False,2935
158713,14458,810000331,2011-01-26,7,08C322,S8221,NaN,P24310,False,2935


## On conserve uniquement les mois de décès

In [33]:
patient['dte_deces']=patient['dte_deces'].dt.to_period('M')

In [34]:
patient.head()

,date_h0,ALD_before,pop,y_nais,BEN_RES_DPT,BEN_SEX_COD,dte_deces,CODE_PATIENT,Mort,Nb_survie
0,2011-11-22,0,1,1949,11,1,NaT,P0,False,2596
6,2013-12-05,0,1,1929,14,2,2015-03,P6,True,456
8,2015-07-02,0,1,1967,54,2,2015-08,P8,True,41
10,2014-11-21,1,0,1932,30,1,2018-12,P10,True,1471
13,2011-05-19,0,1,1959,974,1,NaT,P13,False,2783


In [35]:
count()

Nb patients = 13310
Nb hospit = 158715
Nb conso = 13241


## On conserve uniquement les patients majeurs au moment de leur première hospitalisation

#### Nombre de patients mineurs dans les données

In [36]:
np.unique(((pd.to_datetime(patient.date_h0).apply(lambda x : x.year) - patient.y_nais) < 18), return_counts=True)

(array([False,  True]), array([13220,    90]))

In [37]:
patient = patient[pd.to_datetime(patient.date_h0).apply(lambda x : x.year) - patient.y_nais >= 18]

In [38]:
consommant = consommant[consommant['CODE_PATIENT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [39]:
hospit = hospit[hospit['BEN_NIR_IDT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [40]:
patient

,date_h0,ALD_before,pop,y_nais,BEN_RES_DPT,BEN_SEX_COD,dte_deces,CODE_PATIENT,Mort,Nb_survie
0,2011-11-22,0,1,1949,11,1,NaT,P0,False,2596
6,2013-12-05,0,1,1929,14,2,2015-03,P6,True,456
8,2015-07-02,0,1,1967,54,2,2015-08,P8,True,41
10,2014-11-21,1,0,1932,30,1,2018-12,P10,True,1471
13,2011-05-19,0,1,1959,974,1,NaT,P13,False,2783
...,...,...,...,...,...,...,...,...,...,...
24302,2014-02-08,0,1,1920,29,1,2014-12,P24302,True,318
24303,2012-02-09,0,1,1927,61,1,2017-05,P24303,True,1924
24304,2010-07-27,0,1,1924,999,1,2010-08,P24304,True,5
24308,2011-11-25,0,1,1925,13,2,2013-03,P24308,True,484


In [41]:
count()

Nb patients = 13220
Nb hospit = 157199
Nb conso = 13151


## On retire les patients ayant déjà connu une hospitalisation avant l'étude (pop=0)

In [42]:
patient=patient[patient["pop"]==1].reset_index(drop=True)

In [43]:
consommant = consommant[consommant['CODE_PATIENT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [44]:
hospit = hospit[hospit['BEN_NIR_IDT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [45]:
patient = patient.drop(columns=['pop'])

In [46]:
count()

Nb patients = 10204
Nb hospit = 157199
Nb conso = 10138


## On retire les patients sans consommation depuis plus de 5 ans, non déclarés décédés

In [60]:
patient_non_décédés = patient[pd.isna(patient['dte_deces'].fillna(pd.Period()))]["CODE_PATIENT"]

In [61]:
patients_sans_conso=consommant[(consommant["CODE_PATIENT"].isin(patient_non_décédés))&(consommant['datemax']<'01-01-2014')]['CODE_PATIENT']

In [62]:
patient = patient[~patient["CODE_PATIENT"].isin(patients_sans_conso)].reset_index(drop=True)

In [63]:
consommant = consommant[consommant['CODE_PATIENT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [64]:
hospit = hospit[hospit['BEN_NIR_IDT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [65]:
count()

Nb patients = 10052
Nb hospit = 156336
Nb conso = 9986


## Ajout des insuffisances cardiaques avec état de choc cardiogénique

In [66]:
#Insuffisances cardiaques et états de choc circulatoire
#hospit[hospit['GRG_GHM'].str.startswith('05M09')]

#Insuffisances cardiaques et états de choc cardiogénique (code R57)
#hospit[(hospit['GRG_GHM'].str.startswith('05M09'))&(hospit['DGN_PAL'].str.startswith('R57'))]

#Ajout d'une colonne CHOC
hospit['CHOC']=hospit.index.isin(hospit[(hospit['GRG_GHM'].str.startswith('05M09'))&(hospit['DGN_PAL'].str.startswith('R57'))|(hospit['DGN_REL'].str.startswith('R57'))].index)
hospit['CHOC']=hospit['CHOC'].astype('int')

In [67]:
hospit.CHOC.sum()

478

## Ajout des hémodialyses

In [68]:
hospit['HEMOD']=hospit.index.isin(hospit[(hospit['DGN_PAL'].str.startswith('Z49'))].index)
hospit['HEMOD']=hospit['HEMOD'].astype('int')

In [69]:
hospit['DGN_PAL'].str.startswith('Z49').sum()

51150

## Ajout des chimiothérapies

In [70]:
hospit['CHIMIO']=hospit.index.isin(hospit[(hospit['DGN_PAL'].str.startswith('Z511'))|(hospit['DGN_PAL'].str.startswith('Z512'))].index)
hospit['CHIMIO']=hospit['CHIMIO'].astype('int')

In [71]:
hospit['DGN_PAL'].str.startswith('Z511').sum(), hospit['DGN_PAL'].str.startswith('Z512').sum()

(9819, 1822)

## Ajout des radiothérapies

In [72]:
hospit['Radiotherapie']=hospit.index.isin(hospit[(hospit['DGN_PAL'].str.startswith('Z510'))|(hospit['DGN_REL'].str.startswith('Z510'))].index)
hospit['Radiotherapie']=hospit['Radiotherapie'].astype('int')

In [73]:
hospit.Radiotherapie.sum()

5660

## Ajout des transfusions

In [74]:
hospit['Transfusion']=hospit.index.isin(hospit[(hospit['GRG_GHM'].str.startswith('28Z14Z'))].index)
hospit['Transfusion']=hospit['Transfusion'].astype('int')

In [75]:
hospit.Transfusion.sum()

2291

## Statistiques descriptives sur les datasets

### Hospitalisation

In [58]:
hospit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156336 entries, 0 to 156335
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   RSA_NUM        156336 non-null  object        
 1   ETA_NUM        156336 non-null  object        
 2   EXE_SOI_DTD    156334 non-null  datetime64[ns]
 3   SEJ_NBJ        153403 non-null  Int64         
 4   GRG_GHM        156336 non-null  object        
 5   DGN_PAL        156336 non-null  object        
 6   DGN_REL        67298 non-null   object        
 7   BEN_NIR_IDT    156336 non-null  object        
 8   Mort           156336 non-null  bool          
 9   Nb_survie      156336 non-null  int64         
 10  CHOC           156336 non-null  int64         
 11  HEMOD          156336 non-null  int64         
 12  CHIMIO         156336 non-null  int64         
 13  Radiotherapie  156336 non-null  int64         
 14  Transfusion    156336 non-null  int64         
dtype

In [59]:
profile_hospit = ProfileReport(hospit, title="Hospit Profile Report")

In [60]:
profile_hospit.to_file("profile_hospit.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Consommant

In [61]:
consommant.head()

,_TEMG001,datemax,CODE_PATIENT
0,21769,2019-08-08,P0
1,20150,2015-03-03,P6
2,20312,2015-08-12,P8
3,21769,2019-08-08,P13
4,21787,2019-08-26,P16


In [62]:
consommant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9986 entries, 0 to 9985
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   _TEMG001      9986 non-null   int64         
 1   datemax       9986 non-null   datetime64[ns]
 2   CODE_PATIENT  9986 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 234.2+ KB


In [63]:
profile_conso = ProfileReport(consommant, title="Consommant Profile Report")

In [64]:
profile_conso.to_file("profile_conso.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Patient

In [65]:
patient.head()

,date_h0,ALD_before,y_nais,BEN_RES_DPT,BEN_SEX_COD,dte_deces,CODE_PATIENT,Mort,Nb_survie
0,2011-11-22,0,1949,11,1,NaT,P0,False,2596
1,2013-12-05,0,1929,14,2,2015-03,P6,True,456
2,2015-07-02,0,1967,54,2,2015-08,P8,True,41
3,2011-05-19,0,1959,974,1,NaT,P13,False,2783
4,2010-03-18,0,1947,76,1,NaT,P16,False,3210


In [66]:
patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10052 entries, 0 to 10051
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date_h0       10052 non-null  datetime64[ns]
 1   ALD_before    10052 non-null  int64         
 2   y_nais        10052 non-null  int64         
 3   BEN_RES_DPT   10052 non-null  object        
 4   BEN_SEX_COD   10052 non-null  int64         
 5   dte_deces     6619 non-null   period[M]     
 6   CODE_PATIENT  10052 non-null  object        
 7   Mort          10052 non-null  bool          
 8   Nb_survie     10052 non-null  int64         
dtypes: bool(1), datetime64[ns](1), int64(4), object(2), period[M](1)
memory usage: 638.2+ KB


In [67]:
profile_patient = ProfileReport(patient, title="Patient Profile Report")

In [68]:
profile_patient.to_file("profile_patient.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

####

#### Nombres de patients et d'hospitalisations après processing

In [76]:
#Nombre de patients
patient_init = pd.read_csv("base_patient.csv", sep=';')
print(f"Il y avait {len(patient_init)} patients initialement. Suite à l'ensemble des critères de rejet, il en reste {len(patient)}.")
print(f"On en a enlevé {len(patient_init) - len(patient)}.")

Il y avait 24311 patients initialement. Suite à l'ensemble des critères de rejet, il en reste 10052.
On en a enlevé 14259.


In [77]:
hospit_init = pd.read_csv("base_hospit.csv", sep=';')
print(f"Il y avait {len(hospit_init)} hospitalisations initialement. Suite à l'ensemble des critères de rejet, il en reste {len(hospit)}.")
print(f"On en a enlevé {len(hospit_init) - len(hospit)}.")

Il y avait 226127 hospitalisations initialement. Suite à l'ensemble des critères de rejet, il en reste 156336.
On en a enlevé 69791.


In [78]:
conso_init = pd.read_csv("consommant.csv", sep=';')
print(f"Il y avait {len(conso_init)} consommations initialement. Suite à l'ensemble des critères de rejet, il en reste {len(consommant)}.")
print(f"On en a enlevé {len(conso_init) - len(consommant)}.")

Il y avait 24232 consommations initialement. Suite à l'ensemble des critères de rejet, il en reste 9986.
On en a enlevé 14246.


#### Nombre de patients dialysés

###### Chimiothérapie

In [79]:
hospit_chimio = hospit.loc[hospit.CHIMIO == 1]
nb_patients_chimio = hospit_chimio.BEN_NIR_IDT.nunique()
print(f"Il y a {len(hospit_chimio)} hospitalisations pour des raisons de chimiothérapie.")
print(f"Cela concerne {nb_patients_chimio} patients.")

Il y a 11641 hospitalisations pour des raisons de chimiothérapie.
Cela concerne 916 patients.


###### Hémodialyse

In [80]:
hospit_hemod = hospit.loc[hospit.HEMOD == 1]
nb_patients_hemod = hospit_hemod.BEN_NIR_IDT.nunique()
print(f"Il y a {len(hospit_hemod)} hospitalisations pour des raisons d'hémodialyse.")
print(f"Cela concerne {nb_patients_hemod} patients.")

Il y a 51150 hospitalisations pour des raisons d'hémodialyse.
Cela concerne 343 patients.


###### Radiothérapie

In [81]:
hospit_radiotherapie = hospit.loc[hospit.Radiotherapie == 1]
nb_patients_radiotherapie = hospit_radiotherapie.BEN_NIR_IDT.nunique()
print(f"Il y a {len(hospit_radiotherapie)} hospitalisations pour des raisons de radiothérapie.")
print(f"Cela concerne {nb_patients_radiotherapie} patients.")

Il y a 5660 hospitalisations pour des raisons de radiothérapie.
Cela concerne 302 patients.


###### Transfusions

In [82]:
hospit_transfusion = hospit.loc[hospit.Transfusion == 1]
nb_patients_transfusion = hospit_transfusion.BEN_NIR_IDT.nunique()
print(f"Il y a {len(hospit_transfusion)} hospitalisations pour des raisons de transfusion.")
print(f"Cela concerne {nb_patients_transfusion} patients.")

Il y a 2291 hospitalisations pour des raisons de transfusion.
Cela concerne 382 patients.


###### Hospitalisations autres

In [83]:
hospit_non_chronique = hospit.loc[(hospit.HEMOD == 0) & (hospit.CHIMIO==0) & (hospit.Radiotherapie==0) & (hospit.Transfusion==0)]
nb_patients_non_chronique = hospit_non_chronique.BEN_NIR_IDT.nunique()
print(f"Il y a {len(hospit_non_chronique)} hospitalisations non chroniques.")
print(f"Cela concerne {nb_patients_non_chronique} patients.")
print("Tous les patients de la base ont au moins une hospitalisation qui n'est pas chronique.")
print(f"On a enlevé, en tout, {len(hospit_init) - len(hospit_non_chronique)} hospitalisations.")

Il y a 85594 hospitalisations non chroniques.
Cela concerne 10051 patients.
Tous les patients de la base ont au moins une hospitalisation qui n'est pas chronique.
On a enlevé, en tout, 140533 hospitalisations.


#### Pourcentages de choc parmi les personnes ayant eu des problèmes cardiaques

In [86]:
pct_choc = hospit_non_chronique[hospit_non_chronique.GRG_GHM.str.startswith('05')].CHOC.sum() / len(hospit_non_chronique[hospit_non_chronique.GRG_GHM.str.startswith('05')])*100
print(f"Il y a {round(pct_choc, 2)}% de personnes ayant eu un choc parmi les personnes ayant eu des problèmes cardiaques.")

Il y a 1.93% de personnes ayant eu un choc parmi les personnes ayant eu des problèmes cardiaques.


#### Profiling sur les hospitalisations après processing et pas chroniques

In [77]:
profile_hospit_non_chroniques = ProfileReport(hospit_non_chronique, title="Hospitalisations Non Chroniques")
profile_hospit_non_chroniques.to_file("profile_hospit_non_chr.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Export des bases de données nettoyées

In [78]:
hospit.to_csv("base_full_hospit_anonyme.csv", index=False)

In [79]:
hospit_non_chronique.to_csv("base_hospit_non_chr_anonyme.csv", index=False)

In [80]:
patient.to_csv("patient_anonyme.csv", index=False)

In [81]:
consommant.to_csv("consommant_anonyme.csv", index=False)